In [1]:
import symforce
symforce.set_epsilon_to_symbol()

import cv2 as cv
import sys
sys.path.append("..")
import slam_utils as su
import numpy as np

import symforce.symbolic as sf
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer
from symforce.values import Values

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
img_1 = cv.imread("./1.png")
img_2 = cv.imread("./2.png")
keypoints_1, keypoints_2, matches = su.find_feature_matches(img_1, img_2)
print(f"Number of matches: {len(matches)}")

Max dist: 94.0
Min dist: 4.0
Number of matches: 79


In [3]:
depth1 = cv.imread("./1_depth.png", cv.IMREAD_UNCHANGED)
depth2 = cv.imread("./2_depth.png", cv.IMREAD_UNCHANGED)
K = np.array([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])
# these have to be Nx3 numpy arrays
pts1 = np.empty((0, 3))
pts2 = np.empty((0, 3))
for m in matches:
    d1 = depth1[int(keypoints_1[m.queryIdx].pt[1]),
                int(keypoints_1[m.queryIdx].pt[0])]
    d2 = depth2[int(keypoints_2[m.trainIdx].pt[1]),
                int(keypoints_2[m.trainIdx].pt[0])]
    if d1 == 0 or d2 == 0:
        continue
    dd1 = d1/5000.0
    dd2 = d2/5000.0
    p1 = su.pixel2cam(keypoints_1[m.queryIdx].pt, K)
    p2 = su.pixel2cam(keypoints_2[m.trainIdx].pt, K)
    pts1 = np.vstack([pts1, np.array([p1[0]*dd1, p1[1]*dd1, dd1])])
    pts2 = np.vstack([pts2, np.array([p2[0]*dd2, p2[1]*dd2, dd2])])
print(f"3d-3d pairs: {len(pts1)}")

3d-3d pairs: 72


In [4]:
# Solve it with SVD and numpy first
p1 = np.zeros((3, 1))  # center of mass
p2 = np.zeros((3, 1))
N = pts1.shape[0]
for i in range(N):
    p1 += pts1[i].reshape(3, 1)
    p2 += pts2[i].reshape(3, 1)
p1 /= N
p2 /= N
q1, q2 = np.zeros((N, 3)), np.zeros((N, 3)) # remove the center
for i in range(N):
    q1[i] = pts1[i] - p1.reshape(3,)
    q2[i] = pts2[i] - p2.reshape(3,)

# compute q1*q2^T
W = np.zeros((3, 3))
for i in range(N):
    W += np.outer(q1[i], q2[i])

# SVD on W
U, _, Vt = np.linalg.svd(W)
V = Vt.T

R_ = U @ V.T
if np.linalg.det(R_) < 0:
    V[:, 2] *= -1
    R_ = U @ V.T

t_ = p1.reshape(-1, 1) - R_ @ p2.reshape(-1, 1)

# convert to cv::Mat
R = np.array([[R_[0][0], R_[0][1], R_[0][2]],
                [R_[1][0], R_[1][1], R_[1][2]],
                [R_[2][0], R_[2][1], R_[2][2]]])
t = np.array([t_[0][0], t_[1][0], t_[2][0]])

print("ICP via SVD results: ")
print(f"R = {R}")
print(f"t = {t}")
print(f"R_inv = {np.linalg.inv(R)}")
print(f"t_inv = {np.linalg.inv(-R)@t}")

ICP via SVD results: 
R = [[ 0.99694524  0.05983348 -0.05020111]
 [-0.05932608  0.99817197  0.01153856]
 [ 0.05079973 -0.00852508  0.99867247]]
t = [ 0.14415983 -0.06667848 -0.03009795]
R_inv = [[ 0.99694524 -0.05932608  0.05079973]
 [ 0.05983348  0.99817197 -0.00852508]
 [-0.05020111  0.01153856  0.99867247]]
t_inv = [-0.14614626  0.05767441  0.03806435]


In [5]:
# Now solve it with bundle adjustment
def compute_error(pose: sf.Pose3,
                  point: sf.V3,
                  measurement: sf.V3) -> sf.V3: 
    return measurement - pose * point

# Redo K as a Symforce type
K = sf.M33([[520.9, 0, 325.1], [0, 521.0, 249.7], [0, 0, 1]])

initial_values = Values(
    pose = sf.Pose3(),
    points = [sf.V3(x[0], x[1], x[2]) for x in pts2.tolist()],
    measurements = [sf.V3(x[0], x[1], x[2]) for x in pts1.tolist()],
    epsilon = sf.numeric_epsilon,
)

factors = []
for i in range(len(pts1)):
    factors.append(Factor(
        residual=compute_error,
        keys=["pose", f"points[{i}]", f"measurements[{i}]"],
    ))

In [7]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["pose"],
    debug_stats=True,
    params=Optimizer.Params(iterations=1000)
)
result = optimizer.optimize(initial_values)
pose = result.optimized_values.get('pose').to_homogenous_matrix()
display(pose)

[2023-05-29 12:51:03.370] [info] LM<sym::Optimize> [iter    0] lambda: 1.000e+00, error prev/linear/new: 1.199/0.000/0.910, rel reduction: 0.24145
[2023-05-29 12:51:03.374] [info] LM<sym::Optimize> [iter    1] lambda: 2.500e-01, error prev/linear/new: 0.910/0.000/0.908, rel reduction: 0.00202
[2023-05-29 12:51:03.378] [info] LM<sym::Optimize> [iter    2] lambda: 6.250e-02, error prev/linear/new: 0.908/0.000/0.908, rel reduction: 0.00001
[2023-05-29 12:51:03.381] [info] LM<sym::Optimize> [iter    3] lambda: 1.562e-02, error prev/linear/new: 0.908/0.000/0.908, rel reduction: 0.00000


array([[ 0.99694524,  0.05983406, -0.05020036,  0.14415858],
       [-0.05932665,  0.99817193,  0.01153899, -0.06667927],
       [ 0.05079901, -0.00852552,  0.99867251, -0.03009813],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])